In [ ]:
pip install tensorflow==2.10.1

# Generalizar nombre,tamaño y extension de las imagenes.

In [ ]:
import os
import shutil
from PIL import Image

# Ruta a la carpeta que contiene las carpetas de frutas
ruta_principal = "../../data/train"

# Ruta de la nueva carpeta de salida
ruta_salida = "../../dataset/entrenamiento"

# Tamaño deseado para las imágenes redimensionadas
nuevo_tamaño = (100, 100)

# Crea la nueva carpeta de salida si no existe
os.makedirs(ruta_salida, exist_ok=True)

# Recorre las carpetas de frutas
for carpeta in os.listdir(ruta_principal):
    carpeta_ruta = os.path.join(ruta_principal, carpeta)
    
    # Verifica si es una carpeta
    if os.path.isdir(carpeta_ruta):
        contador = 1

        # Crea una nueva carpeta en la ruta de salida con el nombre de la carpeta actual
        nueva_carpeta_salida = os.path.join(ruta_salida, carpeta)
        os.makedirs(nueva_carpeta_salida, exist_ok=True)
        
        # Recorre los archivos en la carpeta de frutas
        for archivo in os.listdir(carpeta_ruta):
            archivo_ruta = os.path.join(carpeta_ruta, archivo)
            
            # Construye el nuevo nombre de archivo
            nuevo_nombre = f"{carpeta}_{contador}.jpg"
            
            # Renombra y copia el archivo en la nueva carpeta de salida
            shutil.copy(archivo_ruta, os.path.join(nueva_carpeta_salida, nuevo_nombre))
            
            # Redimensiona y convierte la imagen al modo RGB (con manejo de excepciones)
            try:
                imagen = Image.open(os.path.join(nueva_carpeta_salida, nuevo_nombre))
                imagen = imagen.convert("RGB")  # Conversión al modo RGB
                imagen = imagen.resize(nuevo_tamaño)
                imagen.save(os.path.join(nueva_carpeta_salida, nuevo_nombre))
            except Exception as e:
                print(f"Error al procesar {nuevo_nombre}: {str(e)}")
            
            contador += 1


# Generar transformaciones de la imagenes para incrementar los datos

In [1]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import random

# Ruta a la carpeta que contiene las carpetas de frutas
ruta_principal = "../../dataset/entrenamiento"

# Número deseado de imágenes aumentadas por imagen original
num_aumentos = 10  # Puedes ajustar esto

# Configura el generador de imágenes aumentadas
data_generator = ImageDataGenerator(
    brightness_range=(0.5, 1.3),
    rotation_range=30,
    #width_shift_range=0.2,
    #height_shift_range=0.2,
    #shear_range=15,
    #zoom_range=[0.7, 1.4],
    horizontal_flip=True,
    vertical_flip=True
)

# Recorre las carpetas de frutas
for carpeta in os.listdir(ruta_principal):
    carpeta_ruta = os.path.join(ruta_principal, carpeta)
    
    # Verifica si es una carpeta
    if os.path.isdir(carpeta_ruta):
        
        # Recorre los archivos en la carpeta de frutas
        for archivo in os.listdir(carpeta_ruta):
            archivo_ruta = os.path.join(carpeta_ruta, archivo)
            
            # Abre la imagen original
            imagen_original = tf.keras.preprocessing.image.load_img(archivo_ruta)
            
            # Convierte la imagen a un arreglo numpy
            imagen_array = tf.keras.preprocessing.image.img_to_array(imagen_original)
            imagen_array = imagen_array.reshape((1,) + imagen_array.shape)  # Añade una dimensión de lote
            
            # Genera imágenes aumentadas
            contador = 0
            for batch in data_generator.flow(imagen_array, batch_size=1):
                if contador >= num_aumentos:
                    break  # Detén cuando se alcance el número deseado de aumentos
                
                imagen_aumentada = tf.keras.preprocessing.image.array_to_img(batch[0])
                
                # Genera un nombre para la imagen aumentada
                nombre_archivo_aumentado = f"{archivo.split('.')[0]}_{contador}.jpg"
                ruta_salida = os.path.join(carpeta_ruta, nombre_archivo_aumentado)
                
                # Guarda la imagen aumentada en la misma carpeta que la imagen base
                imagen_aumentada.save(ruta_salida)
                
                contador += 1
